# Question answering (QnA)


![](https://lilianweng.github.io/posts/2020-10-29-odqa/QA-summary.png)

## QnA adatbázisok

A [MILQA](https://huggingface.co/datasets/SzegedAI/MILQA) adatbázis 15509 kérdés-válaszból áll a következő információkkal: szöveg (tipikusan kb egy bekezdésnyi), szöveghez hozzátartozó title, egy kérdés, amely a szövegre vonatkozik, illetve a szövegből egy részlet, mely megválaszolja a kérdést, aminek megvan adva a karakter pozíciója is.

Létezik egy nagyobb angol adabázis, az a SQuAD nevet kapta.

Quick guide: https://towardsdatascience.com/the-quick-guide-to-squad-cae08047ebee

The Stanford Question Answering Dataset: https://rajpurkar.github.io/SQuAD-explorer/


## Retrieval
Töltsük le e a MILQA adatbázist.

In [ ]:
import pandas as pd

milqa = pd.read_csv('https://raw.githubusercontent.com/zsozso21/soc_media/main/milqa_short_answers.csv')

milqa.head(10)

 Ebben az fájlban 2083 wikipédáról származó szövegrészlet (*context*) található 15509 darab hozzájuk tarotozó kérdéssel (*question*) és válasszal (*answer*).

 Csökkentsük le az adatbázis méretét, maradjon benne 100 db egyedi szövegrészlet és mindegyikhez 1-1 db kérdés és válasz.

In [2]:
milqa_small = milqa.drop_duplicates("context", keep="first")
milqa_small = milqa_small.sample(100, random_state=42, ignore_index=True)
milqa_small

,Unnamed: 0,id,context,title,question,short_end,short_start,short_text
0,18729,2123,A kockázati tényezők jelenléte elősegíti a kór...,Tüdőgyulladás,Milyen tényezők növelik a tüdőgyulladás kockáz...,225,216,dohányzás
1,10904,1222,2007. június 5-én indult el a mobil verzió pub...,IWiW,Hová vitték az iWiW szerverét 2007-ben?,179,155,a Dataplex adatközpontba
2,15934,1777,"Horthy Miklóst, még hadügyminiszterként, gróf ...",Horthy Miklós (kormányzó),"Milyen pozíciót töltött be Horthy Miklós, amik...",39,20,hadügyminiszterként
3,12508,1377,A párt országos befolyása azonban alig növeked...,Adolf Hitler,Hány tagja volt a nemzetiszocialista pártnak N...,106,78,alig haladta meg a 70 000-et
4,21817,2774,2017 márciusában a Vanity Fairben megjelent eg...,Emma Watson,Melyik újságban jelent meg Emma Watsonról az a...,33,17,a Vanity Fairben
...,...,...,...,...,...,...,...,...
95,20605,2544,"A kincslelet pontos helyének megállapítása, el...",Seuso-kincs,"Mire utal a Seuso-kincs vadásztáljának a ""Pels...",622,577,hogy tulajdonosa a Balaton környékén élhetett
96,8852,1042,"1896-ban az elkészült Ecce Homót, a Krisztus-t...",Munkácsy Mihály,Mikor tartották Budapesten a millenniumi ünnep...,8,0,1896-ban
97,22146,2804,Marosvásárhely első helyi lapja a Székely Elle...,Marosvásárhely,Melyik időszakban létezett a Székely Ellenzék ...,79,63,1898–1920 között
98,22814,2901,Egy iskolai projekt keretein belül felvett egy...,Adele,Adele melyik internetes oldalon lett először s...,109,99,MySpace-re


A retriever modellek feladata, hogy egy adott kérdéshez megkeressék a rá legrelevánsabb szövegrészleteket.

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-small')

input_texts = [
    "query: Hol ered a Duna?",
    "query: Ki vezette az 1521-es szabácsi török támadást?",
    "query: Kik voltak a fehérek az orosz polgárháborúban az 1910-es évek végén?",
    "passage: A Duna Európa leghosszabb folyama az oroszországi Volga után. Németországban, a Fekete-erdőben ered két kis patak, a Breg és a Brigach összefolyásával Donaueschingennél, és innen délkeleti irányban 2850 kilométert tesz meg a Fekete-tengerig. Magyarország egész területe e folyam vízgyűjtőjén terül el, itteni főágának hossza 417 km, ezért az ország vízrajzának meghatározó alkotóeleme. A folyó kialakulása a pliocén korban kezdődött el. A pliocén végén jutott el a Duna a Kisalföldig, ekkor a mai nyugat–kelet irány helyett észak–dél irányban folyt itt. Csak a pleisztocén korban alakult ki a kisalföldi szakasza. A folyó legfiatalabb része a Dobrudzsa nyugati oldalán található dél–észak irányú folyása, amely pusztán a pleisztocén kor végén jött létre. Napjainkban fontos nemzetközi hajóút. A németországi Rajna–Majna–Duna-csatorna 1992-es megépítése óta részét képezi annak a 3500 km-es transzeurópai vízi útnak, amely az Északi-tenger melletti Rotterdamtól a Fekete-tenger melletti Sulináig ér. A Dunán szállított áruk össztömege 1987-ben elérte a 100 millió tonnát.",
    "passage: Szulejmán 1521. május 18-án élete első hadjáratára indult körülbelül ötvenezer fős seregével. Az oszmán hadvezetés megosztott volt a háborús célok tekintetében, végül a ruméliai török sereg Ahmed vezetésével Szabács, míg az anatóliai had Piri Mehmed nagyvezírrel Nándorfehérvár ellen vonult, amit a magyar haderő a határvédelem elavultsága, mozgósítási nehézségek, személyes ellentétek, és a készletek kimerülése miatt nem tudott megvédeni. A török sereg az 1521. évi hadjáratban egy széles rést ütött a délvidéki határszakaszon, ezért egy új védelmi szakaszra volt szükség, hogy a következő években induló offenzívát valamiképp megállítsák. A magyar határvédelem nehéz feladat előtt állt, amit az is jelzett, hogy 1525-ben egy kisebb seregre volt szükség ahhoz, hogy Jajca elszigetelt végvárába megérkezzen az ellátás. A siker kiaknázása török részről azért nem lehetett maximális, mert Szulejmán hátrahagyta az anatóliai seregeket egy esetleges perzsa támadás kivédésére. Ezzel azonban elvesztette azt a lehetőséget, hogy a Kemálpasazáde-krónikában is írt végcélját, Budát elfoglalhassa."
]


In [ ]:
# Embeddingek készítése
embeddings = model.encode(input_texts, normalize_embeddings=True)
embeddings

In [ ]:
# Hasonlóság számítása
similarities = model.similarity(embeddings[:3], embeddings[3:])
similarities

tensor([[0.8551, 0.7482],
        [0.7395, 0.8581],
        [0.7505, 0.7859]])


Készítsünk embeddinget a MILQA adatbázis kérdésiből és szövegrészletiből.

In [ ]:
contexts_embeddings = model.encode(milqa_small.context, normalize_embeddings=True)
questions_embeddings = model.encode(milqa_small.question, normalize_embeddings=True)

In [ ]:
similarities = model.similarity(questions_embeddings, contexts_embeddings)
similarities

tensor([[0.8691, 0.7507, 0.7280,  ..., 0.7433, 0.7289, 0.7682],
        [0.7840, 0.8810, 0.7659,  ..., 0.7533, 0.8169, 0.7880],
        [0.7584, 0.7673, 0.9119,  ..., 0.7601, 0.7597, 0.7655],
        ...,
        [0.7468, 0.7744, 0.8010,  ..., 0.8394, 0.7737, 0.7641],
        [0.7752, 0.8066, 0.7689,  ..., 0.7708, 0.8567, 0.7633],
        [0.7812, 0.7653, 0.7623,  ..., 0.7795, 0.7608, 0.8732]])

In [ ]:
# A kérdésekhez a leghasonlóbb kontextust egyszerűen meghatározhatjuk úgy,
# hogy lekérjük a hasonlóságok maximumának az indexét.

most_similar_contexts = similarities.argmax(axis=1)
most_similar_contexts

tensor([69,  1,  2,  3,  4,  5,  6,  7,  8, 64, 10, 11, 12, 13, 42, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 12, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57,  9, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 37,
        72, 73, 74, 75, 76, 89, 78, 79, 80, 70, 82, 83, 84, 85, 86, 87, 88, 89,
        90, 91, 92, 93, 94, 95, 56, 97, 98, 99])

Azokon a helyeken, ahol a kérdés sorszáma megegyezik a leghasonlóbb kontextus sorszámával a retriever modell jó megoldást talált meg, értékeljük ki ez hány %-ban volt igaz.

In [ ]:
correct = 0

for i in range(len(most_similar_contexts)):
  if most_similar_contexts[i] == i:
    correct += 1
  else:
    print("Hibás találat:", i, "=>", int(most_similar_contexts[i]))

print("A rendszer hatékonysága:", correct/len(most_similar_contexts))

Hibás találat: 0 => 69
Hibás találat: 9 => 64
Hibás találat: 14 => 42
Hibás találat: 37 => 12
Hibás találat: 58 => 9
Hibás találat: 71 => 37
Hibás találat: 77 => 89
Hibás találat: 81 => 70
Hibás találat: 96 => 56
A rendszer hatékonysága: 0.91


Fontos megjegyezni, hogy minél több szövegrészletünk van, annál nehezebb a feladat.

Vizsgáljuk meg a hibás eseteket:

In [ ]:
import textwrap

question_id = 0

print("Kérdés: {}".format(milqa_small.question[question_id]))
print()
print(textwrap.fill("Helyes válasz: {}".format(milqa_small.context[question_id])))
print()
most_similar_context = milqa_small.context[int(most_similar_contexts[0])]
print(textwrap.fill("Leghasonlóbb szövegrészlet: {}".format(most_similar_context)))

Kérdés: Milyen tényezők növelik a tüdőgyulladás kockázatát?

Helyes válasz: A kockázati tényezők jelenléte elősegíti a kórokozók
elszaporodását. Összefüggés mutatkozik az alkoholizmus és a
Streptococcus pneumoniae, az anaerob kórokozók és a Mycobacterium
tuberculosis elszaporodása között; a dohányzás elősegíti a
Streptococcus pneumoniae, a Haemophilus influenzae, a Moraxella
catarrhalis és a Legionella pneumophila fertőzését. A madarakkal való
kontaktus a Chlamydia psittacival, a háziállatokkal való kontaktus a
Coxiella burnettivel, a gyomortartalom aspiratiója az anaerob
kórokozókkal, a cisztás fibrózis pedig a Pseudomonas aeruginosaval és
a Staphylococcus aureusszal függ össze. A Streptococcus pneumoniae
gyakoribb télen, és azoknál kell erre gondolni, akik nagy mennyiségű
anaerob kórokozót aspiráltak.

Leghasonlóbb szövegrészlet: A tüdőgyulladás évente körülbelül 450
millió embert érint – ami a világ összlakosságának 7%-át jelenti – és
körülbelül 4 millió halálos áldozata van. Bár a 

In [ ]:
milqa_small.loc[69, 'title']


'Tüdőgyulladás'

## Extraktív QnA
Extraktív QnA-ra használjunk egy előre betanított modellt, amely a hubert modell finomhangolt változata.

Ez a modell megtalálható a [HuggingFace](https://huggingface.co/ZTamas/hubert-qa_milqa)-n, ahol mellékelve van egy demó felület is.

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model = "ZTamas/hubert-qa-milqa",
    tokenizer = "ZTamas/hubert-qa-milqa",
    device=device)

In [ ]:
context1 = """
        Az írott sajtó is természetesen megemlékezett a holdra szállásról, a legnagyobb magyar napilap, a Népszabadság például címlapon
        közölte a hírt, illetve belső oldalain nagyobb cikkben írta le az eseményeket, azonban az ilyen kivételes alkalmakhoz alkalmazott
        piros betűs fejléc ezúttal elmaradt. A lapok visszafogottan, tárgyszerűen és szakmailag nagyon korrekten tudósítottak, de mindenféle
        különösebb – az esemény jelentőségéhez egyébként illő – lelkesedés nélkül. Emellett némileg megpróbálták ellensúlyozni a szovjet vereséget
        azzal, hogy a Luna–15 szondának a holdra szállással egy időben futtatott, de lezuhanással végződött küldetését „sikeresnek" tüntették fel,
        és jelentőségét szélsőségesen eltúlozták, továbbá szerkesztőségi cikkben finom hangulatkeltéssel utaltak a rakétatechnika atomfegyverkénti
        felhasználására is, rombolva az amerikaiak által gondosan épített nonmilitarista képet.
        """
question1 = 'Hova szállt a Luna-15?'
gs_answer = 'lezuhanással'

In [ ]:
prediction1 = qa_pipeline({'context': context1, 'question': question1})
prediction1

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.21746662259101868, 'start': 603, 'end': 611, 'answer': 'a holdra'}

In [ ]:
prediction1['answer'] == gs_answer

False

In [ ]:
context2 = 'Belgorod elfoglalása után Harkov, majd Kijev felé törhettek előre. Eközben (augusztus 7-én) megkezdődött a harc Szmolenszk városáért. Októberben a város környékét is visszafoglalták, így már egészen Fehéroroszországig jutottak. 1943 októberében átszervezték a szovjet haderőt. Délen négy új frontot hoztak létre, amelyek az 1–4. Ukrán Front elnevezést kapták. Az ezek alá rendelt hadtestek novemberre elvágták a Krímben állomásozó német alakulatokat, november 6-án ugyanis már Kijev is szovjet kézre került. A december 24-én kezdődött új offenzíva eredményeként az év utolsó napján Zsitomir is elesett.'
question2 = 'A szovjetek mikor szerezték vissza Kijevet a németektől a 2. világháborúban?'
gs_answer2 = 'november 6-án'

In [ ]:
prediction2 = qa_pipeline({'context': context2, 'question': question2})
prediction2

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'score': 0.94158536195755,
 'start': 451,
 'end': 464,
 'answer': 'november 6-án'}

In [ ]:
prediction2['answer'] == gs_answer2

True

### Kiértékelő adatbázis előkészítése

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm,trange
from collections import Counter

In [ ]:
def eval_transformer(line, milqa_corpus):
    context = milqa_corpus.loc[line, 'context']
    question_text = milqa_corpus.loc[line, 'question']
    prediction = qa_pipeline({'context': context, 'question': question_text})

    return prediction['answer'] == milqa_corpus.loc[line, 'short_text'];


In [ ]:
tf_match = []

for i in trange(len(milqa_small)):
    tf_match.append(eval_transformer(i, milqa_small))

  0%|          | 0/100 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
Counter(tf_match)[True]/(Counter(tf_match)[False]+Counter(tf_match)[True])

0.69

## RAG - Retrieval Augmented Generation (Absztraktív QnA)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("unsloth/Llama-3.2-1B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = model.to(device)

Using device: cuda


In [ ]:
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information above I want you to create a short answer to the query in a crisp manner, in case case you don't know the answer say 'Sajnos nem tudok a kérdésre válasozlni'.\n"
    "Always answer in Hungarian.\n"
    "Query: {query_str}\n"
    "Answer: "
    )

In [ ]:
def generate_answer(context, question):
    prompt = qa_prompt_tmpl_str.format(context_str=context, query_str=question)
    print(textwrap.fill("Prompt: {}".format(prompt), replace_whitespace=False, drop_whitespace=False))
    print()

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.1, do_sample=True
        )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # remove the prompt from the output
    answer = answer.replace(prompt, "")
    answer = answer.strip()

    return answer


In [ ]:
answer = generate_answer(milqa_small.context[0], milqa_small.question[0])
print(textwrap.fill(answer))

Prompt: Context information is below.
---------------------
A 
kockázati tényezők jelenléte elősegíti a kórokozók elszaporodását. 
Összefüggés mutatkozik az alkoholizmus és a Streptococcus pneumoniae, 
az anaerob kórokozók és a Mycobacterium tuberculosis elszaporodása 
között; a dohányzás elősegíti a Streptococcus pneumoniae, a 
Haemophilus influenzae, a Moraxella catarrhalis és a Legionella 
pneumophila fertőzését. A madarakkal való kontaktus a Chlamydia 
psittacival, a háziállatokkal való kontaktus a Coxiella burnettivel, a
 gyomortartalom aspiratiója az anaerob kórokozókkal, a cisztás 
fibrózis pedig a Pseudomonas aeruginosaval és a Staphylococcus 
aureusszal függ össze. A Streptococcus pneumoniae gyakoribb télen, és 
azoknál kell erre gondolni, akik nagy mennyiségű anaerob kórokozót 
aspiráltak.

---------------------
Given the context information above
 I want you to create a short answer to the query in a crisp manner, 
in case case you don't know the answer say 'Sajnos nem tudok